# 睿抗 2025 智海人工智能算法应用赛
**语音情绪识别**：使用wav2vec2 预训练模型 + 分类头
- 核心思想：
- 使用预训练模型作为特征提取器，效果显著优于手工提取
- 单模线上 0.87 3折 0.89 5折推理超时，但应该大于0.9 推理没做太多优化
  

In [1]:
import torch
import random
import numpy as np

In [2]:
def set_seed(seed=42):
    random.seed(seed)  # Python 内置随机数
    np.random.seed(seed)  # NumPy 随机数
    torch.manual_seed(seed)  # CPU 上的随机数
    torch.cuda.manual_seed(seed)  # GPU 上的随机数
    torch.cuda.manual_seed_all(seed)  # 多 GPU 情况下的随机数
    torch.backends.cudnn.deterministic = True  # 确保每次卷积结果一致
    torch.backends.cudnn.benchmark = False     # 禁用自动优化

set_seed()

In [3]:
import torchaudio.transforms as T
import torchaudio

In [ ]:
import os, glob, random, torch, torchaudio
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch_audiomentations import Compose, AddColoredNoise, PitchShift, Gain

class WavDataset(Dataset):
    """
    数据读取
    数据增强
    数据预处理
    """
    def __init__(
        self,
        root: str,
        sr_t: int = 16_000,
        max_sec: int = 3,
        training: bool = True,
        aug_prob: float = 0.4,        # 整体执行增强的概率
    ):
        self.wavs, self.lbl = [], []
        for idx, emo in enumerate(sorted(os.listdir(root))):
            for f in glob.glob(f"{root}/{emo}/*.wav"):
                self.wavs.append(f)
                self.lbl.append(idx)

        # 基本配置
        self.sr_t = sr_t
        self.max_len = max_sec * sr_t
        self.training = training
        self.aug_prob = aug_prob

        # -------- 数据增强管道，仅训练用 --------
        if self.training:
            self.augment = Compose(
                transforms=[
                    PitchShift(
                        min_transpose_semitones=-1,
                        max_transpose_semitones=1,
                        sample_rate=self.sr_t,       # ← 必填
                        output_type="tensor",        # ← 推荐手动写死
                        p=0.5,
                    ),
                    Gain(
                        min_gain_in_db=-3,
                        max_gain_in_db=3,
                        sample_rate=self.sr_t,
                        output_type="tensor",
                        p=0.5,
                    ),
                ],
                shuffle=True,
                p=self.aug_prob,
                output_type="tensor",               # 给 Compose 也加一份，防未来破坏性变更
            )
        else:
            self.augment = None

    def __len__(self):
        return len(self.wavs)

    def __getitem__(self, idx):
        wav, sr = torchaudio.load(self.wavs[idx])        # [1, L]
        if sr != self.sr_t:
            wav = torchaudio.functional.resample(wav, sr, self.sr_t)

        wav = wav.mean(0)                                # → [L]
        L = wav.numel()

        # --- 随机截取 / 填充 ---
        if L > self.max_len:
            start = random.randint(0, L - self.max_len) if self.training else 0
            wav = wav[start : start + self.max_len]
        else:
            wav = F.pad(wav, (0, self.max_len - L))

        # --- 数据增强 ---
        if self.augment is not None:
            # torch-audiomentations 期望形状 [B, 1, T]
            wav = self.augment(wav.unsqueeze(0).unsqueeze(0)).squeeze()

        return wav, self.lbl[idx]

## 模型构建

这里的分类头加了一个自注意力池化，其实就是加了两个线性层

In [5]:
import torch, torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor

PRETRAINED = "./wav2vec2-base"          # 可换成中文/多语权重
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(PRETRAINED)

class AttentivePool(nn.Module):
    """自注意力池化：权重 = softmax(Linear(tanh(Linear)))"""
    def __init__(self, hidden):
        super().__init__()
        self.linear1 = nn.Linear(hidden, hidden // 2)
        self.linear2 = nn.Linear(hidden // 2, 1)

    def forward(self, h):            # h: [B, T, C]
        a = torch.tanh(self.linear1(h))
        a = self.linear2(a).squeeze(-1)          # [B, T]
        a = torch.softmax(a, dim=1).unsqueeze(-1)
        return (h * a).sum(1)                    # [B, C]

class HF_Wav2Vec2SER(nn.Module):
    def __init__(self, num_cls=5, freeze_feat=True, pool="attn"):
        super().__init__()
        self.encoder = Wav2Vec2Model.from_pretrained(PRETRAINED)
        if freeze_feat:
            self.encoder.feature_extractor.requires_grad_(False)

        hid = self.encoder.config.hidden_size           # 768

        # ------- 池化层选择 -------
        if pool == "mean":
            self.pool = lambda h: h.mean(1)
        elif pool == "stat":
            self.pool = lambda h: torch.cat([h.mean(1), h.std(1)], dim=-1)
            hid *= 2                                     # 因为拼接 mean+std
        else:  # "attn"
            self.pool = AttentivePool(hid)

        # ------- 分类头 -------
        self.classifier = nn.Sequential(
            nn.LayerNorm(hid),
            nn.Linear(hid, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_cls)
        )

    def forward(self, wav, attn_mask):
        h = self.encoder(wav, attention_mask=attn_mask,
                         return_dict=True).last_hidden_state   # [B,T,C]
        x = self.pool(h)
        return self.classifier(x)

In [6]:
def collate_hf(batch):
    wavs, labels = zip(*batch)          # wavs: List[Tensor(L)]
    inputs = feature_extractor(
        [w.numpy() for w in wavs],
        sampling_rate=16_000,
        padding=True,
        return_tensors="pt",
        return_attention_mask=True        # 显式要求生成 mask
    )
    return inputs.input_values, inputs.attention_mask, torch.tensor(labels)

## 数据处理

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import torch.nn.functional as F

data_path = 'datasets/67fc7ccbb88b01da6626732d-momodel/train/'
dataset = WavDataset(data_path)

all_indices = list(range(len(dataset)))
all_labels = dataset.lbl

train_idx, val_idx = train_test_split(
    all_indices,
    test_size=0.2,
    stratify=all_labels,
    shuffle=True,
    random_state=42
)

train_set = Subset(dataset, train_idx)
val_set   = Subset(dataset, val_idx)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_hf)
val_loader   = DataLoader(val_set,   batch_size=32, shuffle=False, collate_fn=collate_hf)

## 模型训练
这里给出三种训练方式，综合自己的算力资源，时间来判断，通常先使用hold-out验证代码（结构、超参数等等），没问题了再运行全量或者五折

In [8]:
train_mode = 'fivefold' # 'holdout' # 'full' 'fivefold'

### hold-out 

In [9]:
import torch, torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import torch.nn.functional as F

if train_mode == 'holdout': # 'fivefold'
    # ------------------- 数据处理 ---------------------
    data_path = 'datasets/67fc7ccbb88b01da6626732d-momodel/train/'
    dataset = WavDataset(data_path)
    
    all_indices = list(range(len(dataset)))
    all_labels = dataset.lbl
    
    train_idx, val_idx = train_test_split(
        all_indices,
        test_size=0.2,
        stratify=all_labels,
        shuffle=True,
        random_state=42
    )
    
    train_set = Subset(dataset, train_idx)
    val_set   = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_hf)
    val_loader   = DataLoader(val_set,   batch_size=32, shuffle=False, collate_fn=collate_hf)
    
    # ------------------- 模型训练 ---------------------
    device  = "cuda" if torch.cuda.is_available() else "cpu"
    model   = HF_Wav2Vec2SER(num_cls=5, freeze_feat=True).to(device)
        
    EPOCHS   = 20
    patience = 10              # 早停耐心
    best_f1  = 0.0
    wait     = 0
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-5)
    steps_per_epoch = len(train_loader)         # ← 每个 epoch 的 batch 数
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=5e-5,             # 峰值 LR，可比原 base lr 高 1.5~2 倍
        epochs=EPOCHS,
        steps_per_epoch=steps_per_epoch,
        pct_start=0.3,           # 前 30% 逐步升 LR
        anneal_strategy='cos',   # 后 70% 余弦下降
        div_factor=25,           # 初始 lr = max_lr / 25
        final_div_factor=1e4     # 末 lr = max_lr / 1e4
    )
    scaler    = torch.amp.GradScaler()

    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        total_loss = 0
        prog = tqdm(train_loader, desc=f"Epoch {epoch:02d} [Train]", leave=False)
    
        for wav, attn, y in prog:
            wav, attn, y = wav.to(device), attn.to(device), y.to(device)
    
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast(device_type=device):
                logits = model(wav, attn)
                loss   = F.cross_entropy(logits, y)
    
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
    
            scheduler.step()                 # ← 每个 batch 调用
            total_loss += loss.item()
            prog.set_postfix(
                loss=f"{loss.item():.4f}",
                lr=f"{scheduler.get_last_lr()[0]:.2e}"
            )
    
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch:02d} | Train Loss: {avg_loss:.4f}")
    
        # -------- 验证 -------- #
        model.eval()
        va_pred, va_lab = [], []
        with torch.no_grad(), torch.amp.autocast(device_type=device):
            for wav, attn, y in tqdm(val_loader, desc=f"Epoch {epoch:02d} [Eval]", leave=False):
                wav, attn = wav.to(device), attn.to(device)
                out = model(wav, attn)
                va_pred += out.argmax(1).cpu().tolist()
                va_lab  += y.tolist()
    
        f1 = f1_score(va_lab, va_pred, average="micro")
        print(f"Epoch {epoch:02d} | Val F1: {f1:.4f} | Best: {best_f1:.4f}")
    
        # -------- 早停 & 保存 -------- #
        if f1 > best_f1:
            best_f1 = f1
            wait = 0
            torch.save(model.state_dict(), "best_w2w_model_1.pt")
            print("New best model saved!")
        else:
            wait += 1
            print(f"No improve ({wait}/{patience})")
    
        if wait >= patience:
            ## 3s 3e-5
            # 0.8647
            print(f"\n Early-stopping triggered! Best Val F1 = {best_f1:.4f}")
            break

### 全量训练

In [10]:
import torch, torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import torch.nn.functional as F

if train_mode == 'full': # 'fivefold'
    # ------------------- 数据处理 ---------------------
    data_path = 'datasets/67fc7ccbb88b01da6626732d-momodel/train/'
    dataset = WavDataset(data_path)
    
    all_indices = list(range(len(dataset)))
    all_labels = dataset.lbl
    
    train_idx, val_idx = train_test_split(
        all_indices,
        test_size=0.2,
        stratify=all_labels,
        shuffle=True,
        random_state=42
    )
    
    train_set = Subset(dataset, train_idx)
    val_set   = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_hf)
    val_loader   = DataLoader(val_set,   batch_size=32, shuffle=False, collate_fn=collate_hf)
    
    # ------------------- 模型训练 ---------------------
    device  = "cuda" if torch.cuda.is_available() else "cpu"
    model   = HF_Wav2Vec2SER(num_cls=5, freeze_feat=True).to(device)
    
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=4e-5)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=5e-5,             # 峰值 LR，可比原 base lr 高 1.5~2 倍
        epochs=EPOCHS,
        steps_per_epoch=steps_per_epoch,
        pct_start=0.3,           # 前 30% 逐步升 LR
        anneal_strategy='cos',   # 后 70% 余弦下降
        div_factor=25,           # 初始 lr = max_lr / 25
        final_div_factor=1e4     # 末 lr = max_lr / 1e4
    )
    scaler    = torch.amp.GradScaler()
    
    EPOCHS   = 50
    patience = 10              # 早停耐心
    best_f1  = 0.0
    wait     = 0
    
    for epoch in range(1, EPOCHS + 1):
        # -------- 训练 -------- #
        model.train()
        total_loss = 0
        prog = tqdm(train_loader, desc=f"Epoch {epoch:02d} [Train]", leave=False)
        for wav, attn, y in prog:
            wav, attn, y = wav.to(device), attn.to(device), y.to(device)
    
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast(device_type=device):
                logits = model(wav, attn)
                loss   = F.cross_entropy(logits, y)
    
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
    
            total_loss += loss.item()
            prog.set_postfix(loss=f"{loss.item():.4f}")
    
        scheduler.step()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch:02d} | Train Loss: {avg_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")
    
        # -------- 验证 -------- #
        model.eval()
        va_pred, va_lab = [], []
        with torch.no_grad(), torch.amp.autocast(device_type=device):
            for wav, attn, y in tqdm(val_loader, desc=f"Epoch {epoch:02d} [Eval]", leave=False):
                wav, attn = wav.to(device), attn.to(device)
                out = model(wav, attn)
                va_pred += out.argmax(1).cpu().tolist()
                va_lab  += y.tolist()
    
        f1 = f1_score(va_lab, va_pred, average="micro")
        print(f"Epoch {epoch:02d} | Val F1: {f1:.4f} | Best: {best_f1:.4f}")
    
        # -------- 早停 & 保存 -------- #
        if f1 > best_f1:
            best_f1 = f1
            wait = 0
            torch.save(model.state_dict(), "best_w2w_model.pt")
            print("New best model saved!")
        else:
            wait += 1
            print(f"No improve ({wait}/{patience})")
    
        if wait >= patience:
            print(f"\n Early-stopping triggered! Best Val F1 = {best_f1:.4f}")
            break

### 5折训练

In [11]:
import os, datetime, random, torch, torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np

train_mode = 'fivefold'
if train_mode == 'fivefold':  # 'fivefold' 
    # ========== 日志 ==========
    log_path = "train_log.txt"
    def log(msg):
        ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        line = f"{ts} | {msg}"
        print(line)
        with open(log_path, "a") as f:
            f.write(line + "\n")
    
    # ========== 数据 ==========
    root      = "datasets/67fc7ccbb88b01da6626732d-momodel/train/"
    full_ds   = WavDataset(root)
    indices   = np.arange(len(full_ds))
    labels    = np.array(full_ds.lbl)
    
    batch_sz  = 32
    device    = "cuda" if torch.cuda.is_available() else "cpu"
    n_folds   = 5
    max_epochs= 50
    patience  = 10
    
    # OOF 容器
    oof_pred  = np.zeros(len(full_ds), dtype=int)
    oof_label = labels.copy()
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    log(f"===== 5-Fold Training start — total samples {len(full_ds)} =====")
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(indices, labels), 1):
        log(f"\n=== Fold {fold}/{n_folds} | train {len(tr_idx)} | val {len(va_idx)} ===")
    
        tr_ds, va_ds = Subset(full_ds, tr_idx), Subset(full_ds, va_idx)
        train_loader = DataLoader(tr_ds, batch_sz, True,  num_workers=4,
                              collate_fn=collate_hf, drop_last=True)
        val_loader   = DataLoader(va_ds, batch_sz, False, num_workers=4,
                                  collate_fn=collate_hf)
    
        model  = HF_Wav2Vec2SER(num_cls=5, freeze_feat=True).to(device)
        optim  = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                                   lr=3e-5, weight_decay=1e-4)
    
        steps_per_epoch = len(train_loader)
        sched  = torch.optim.lr_scheduler.OneCycleLR(
            optim,
            max_lr=5e-5,                 # 峰值 LR
            epochs=max_epochs,
            steps_per_epoch=steps_per_epoch,
            pct_start=0.30,
            div_factor=25,
            final_div_factor=1e4,
            anneal_strategy='cos'
        )
        scaler = torch.amp.GradScaler()
    
        best_f1, wait = 0.0, 0
        for epoch in range(1, max_epochs + 1):
            # -------- Train --------
            model.train(); total_loss = 0
            for wav, attn, y in tqdm(train_loader,
                                     desc=f"F{fold} E{epoch} [Train]", leave=False):
                wav, attn, y = wav.to(device), attn.to(device), y.to(device)
                optim.zero_grad(set_to_none=True)
                with torch.amp.autocast(device_type=device):
                    loss = F.cross_entropy(model(wav, attn), y)
                scaler.scale(loss).backward()
                scaler.step(optim); scaler.update()
    
                sched.step()                     # ← 每个 batch 更新 LR
                total_loss += loss.item()
    
            avg_loss = total_loss / len(train_loader)
    
            # -------- Eval --------
            model.eval(); va_pred = []
            with torch.no_grad(), torch.amp.autocast(device_type=device):
                for wav, attn, y in val_loader:
                    logits = model(wav.to(device), attn.to(device))
                    va_pred += logits.argmax(1).cpu().tolist()
            f1 = f1_score(labels[va_idx], va_pred, average="micro")
            log(f"Fold {fold} Ep{epoch:02d} |Train loss {avg_loss:.4f} "
                f"| Valid F1 {f1:.4f} | best F1 {best_f1:.4f} "
                f"| LR {sched.get_last_lr()[0]:.2e}")
    
            # -------- Early-stop & Save --------
            if f1 > best_f1:
                best_f1, wait = f1, 0
                torch.save(model.state_dict(), f"wav2vec2_fold{fold}.pt")
                log("saved best model of this fold")
            else:
                wait += 1
            if wait >= patience:
                log("early-stop")
                break
    
        # 填 OOF
        oof_pred[va_idx] = va_pred
    
    # ===== 计算整体 OOF F1 =====
    oof_f1 = f1_score(oof_label, oof_pred, average="micro")
    log(f"\n Five-fold overall OOF F1 = {oof_f1:.4f}")

2025-06-27 16:19:18 | ===== 5-Fold Training start — total samples 7756 =====
2025-06-27 16:19:18 | 
=== Fold 1/5 | train 6204 | val 1552 ===


/root/miniconda3/lib/python3.12/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
                                                                

2025-06-27 16:20:01 | Fold 1 Ep01 |Train loss 1.5265 | Valid F1 0.4195 | best F1 0.0000 | LR 2.52e-06
2025-06-27 16:20:02 | saved best model of this fold


2025-06-27 16:20:45 | Fold 1 Ep02 |Train loss 1.2528 | Valid F1 0.5876 | best F1 0.4195 | LR 4.08e-06
2025-06-27 16:20:45 | saved best model of this fold


2025-06-27 16:21:29 | Fold 1 Ep03 |Train loss 0.8796 | Valid F1 0.6978 | best F1 0.5876 | LR 6.59e-06
2025-06-27 16:21:29 | saved best model of this fold


...

2025-06-27 18:37:23 | Fold 5 Ep33 |Train loss 0.0293 | Valid F1 0.8672 | best F1 0.8698 | LR 2.39e-05


2025-06-27 18:37:51 | Fold 5 Ep34 |Train loss 0.0185 | Valid F1 0.8562 | best F1 0.8698 | LR 2.16e-05


2025-06-27 18:38:19 | Fold 5 Ep35 |Train loss 0.0207 | Valid F1 0.8549 | best F1 0.8698 | LR 1.94e-05


2025-06-27 18:38:48 | Fold 5 Ep36 |Train loss 0.0162 | Valid F1 0.8640 | best F1 0.8698 | LR 1.73e-05


2025-06-27 18:39:18 | Fold 5 Ep37 |Train loss 0.0092 | Valid F1 0.8652 | best F1 0.8698 | LR 1.52e-05
2025-06-27 18:39:18 | early-stop
2025-06-27 18:39:18 | 
 Five-fold overall OOF F1 = 0.8635


## 7. 评分

In [ ]:
import torch, torchaudio, torch.nn.functional as F, random
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2FeatureExtractor

label_names = ['anger', 'fear', 'happy', 'neutral', 'sad']
PRETRAINED_DIR = "./wav2vec2-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(PRETRAINED_DIR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- 模型结构 ----------
class HF_Wav2Vec2SER(torch.nn.Module):
    def __init__(self, num_cls=5, freeze_feat=True, pool="mean"):
        super().__init__()
        from transformers import Wav2Vec2Model
        self.encoder = Wav2Vec2Model.from_pretrained(PRETRAINED_DIR)
        if freeze_feat:
            self.encoder.feature_extractor.requires_grad_(False)
        hid = self.encoder.config.hidden_size
        self.pool = (lambda h: h.mean(1)) if pool == "mean" else (lambda h: h[:, 0])
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hid, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(128, num_cls)
        )

    def forward(self, wav, attn_mask):
        h = self.encoder(wav, attention_mask=attn_mask, return_dict=True
                        ).last_hidden_state         # [B,T,C]
        x = self.pool(h)                             # [B,C]
        return self.classifier(x)                    # [B,num_cls]

# ---------- 加载5折模型 ----------
models = []
for i in tqdm(range(1, 6)):
    model = HF_Wav2Vec2SER(num_cls=len(label_names)).to(device)
    model.load_state_dict(torch.load(f"model_fold{i}.pt", map_location=device), strict=True)
    model.eval()
    models.append(model)

# ---------- 推理用 Dataset ----------
class InferenceDataset(Dataset):
    def __init__(self, waves, srs, sr_t=44_100, max_sec=3, training=False):
        assert len(waves) == len(srs)
        self.waves, self.srs = waves, srs
        self.sr_t, self.max_len = sr_t, max_sec * sr_t
        self.training = training
        
    def __len__(self): return len(self.waves)

    def __getitem__(self, idx):
        w, sr = self.waves[idx], self.srs[idx]
        if sr != self.sr_t:
            w = torchaudio.functional.resample(w, sr, self.sr_t)
        if w.dim() == 2 and w.size(0) > 1:
            w = w.mean(0)
        L = w.numel()
        if L > self.max_len:
            s = random.randint(0, L - self.max_len) if self.training else 0
            w = w[s: s + self.max_len]
        else:
            w = F.pad(w, (0, self.max_len - L))
        return w

def collate_fn(batch):
    inputs = feature_extractor(
        [w.numpy() for w in batch],
        sampling_rate=16_000,
        padding=True,
        return_tensors="pt",
        return_attention_mask=True
    )
    return inputs.input_values, inputs.attention_mask

# ---------- 集成推理函数 ----------
@torch.no_grad()
def predict(audio: torch.Tensor, sr: int) -> str:
    if audio.dim() == 2 and audio.size(0) == 1:
        audio = audio.squeeze(0)

    ds     = InferenceDataset([audio], [sr])
    loader = DataLoader(ds, batch_size=1, collate_fn=collate_fn)
    wav, attn = next(iter(loader))
    wav, attn = wav.to(device), attn.to(device)

    logits_sum = torch.zeros((1, len(label_names)), device=device)
    for model in models:
        logits_sum += model(wav, attn)

    pred_idx = logits_sum.argmax(1).item()
    return label_names[pred_idx]

In [ ]:
# ---------- 测试 ----------
wav, sr = torchaudio.load('./datasets/67fc7ccbb88b01da6626732d-momodel/train/happy/22.wav')
print("Pred emotion:", predict(wav, sr))